<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 3

### Regression and Classification with the Ames Housing Data

---

You have just joined a new "full stack" real estate company in Ames, Iowa. The strategy of the firm is two-fold:
- Own the entire process from the purchase of the land all the way to sale of the house, and anything in between.
- Use statistical analysis to optimize investment and maximize return.

The company is still small, and though investment is substantial the short-term goals of the company are more oriented towards purchasing existing houses and flipping them as opposed to constructing entirely new houses. That being said, the company has access to a large construction workforce operating at rock-bottom prices.

This project uses the [Ames housing data recently made available on kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques).

## Libraries Import  

<br/><br/>

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [3]:
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import plot_confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import plot_roc_curve, RocCurveDisplay, roc_auc_score
from sklearn.metrics import plot_precision_recall_curve, PrecisionRecallDisplay, average_precision_score
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [4]:
# run to avoid warnings
import warnings
warnings.filterwarnings('ignore')

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Estimating the value of homes from fixed characteristics.

---

Your superiors have outlined this year's strategy for the company:
1. Develop an algorithm to reliably estimate the value of residential houses based on *fixed* characteristics.
2. Identify characteristics of houses that the company can cost-effectively change/renovate with their construction team.
3. Evaluate the mean dollar value of different renovations.

Then we can use that to buy houses that are likely to sell for more than the cost of the purchase plus renovations.

Your first job is to tackle #1. You have a dataset of housing sale data with a huge amount of features identifying different aspects of the house. The full description of the data features can be found in a separate file:

    housing.csv
    data_description.txt
    
You need to build a reliable estimator for the price of the house given characteristics of the house that cannot be renovated. Some examples include:
- The neighborhood
- Square feet
- Bedrooms, bathrooms
- Basement and garage space

and many more. 

Some examples of things that **ARE renovateable:**
- Roof and exterior features
- "Quality" metrics, such as kitchen quality
- "Condition" metrics, such as condition of garage
- Heating and electrical components

and generally anything you deem can be modified without having to undergo major construction on the house.

---

**Your goals:**
1. Perform any cleaning, feature engineering, and EDA you deem necessary.
- Be sure to remove any houses that are not residential from the dataset.
- Identify **fixed** features that can predict price.
- Train a model on pre-2010 data and evaluate its performance on the 2010 houses.
- Characterize your model. How well does it perform? What are the best estimates of price?

> **Note:** The EDA and feature engineering component to this project is not trivial! Be sure to always think critically and creatively. Justify your actions! Use the data description file!

In [5]:
# load the data
house = pd.read_csv('datasets/housing.csv')

In [6]:
# look at the dataset
house.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
# check the shape
house.shape

(1460, 81)

In [8]:
# check the dataset info
house.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [9]:
# check the statistical summary
house.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [10]:
# check for duplicate rows
house.duplicated().sum()

0

In [11]:
# drop Id column
house.drop(columns='Id', inplace=True)

In [12]:
# build a data dictionary which includes description, mutability and variable type for all features
data_dict = {'MSSubClass':['Identifies the type of dwelling involved in the sale', 'fixed', 'nominal'],
             'MSZoning':['Identifies the general zoning classification of the sale', 'fixed', 'nominal'],
             'LotFrontage':['Linear feet of street connected to property', 'fixed', 'continuous'],
             'LotArea':['Lot size in square feet', 'fixed', 'continuous'],
             'Street':['Type of road access to property', 'mutable', 'nominal'],
             'Alley':['Type of alley access to property', 'mutable', 'nominal'], 
             'LotShape':['General shape of property', 'fixed', 'nominal'], 
             'LandContour':['Flatness of the property', 'fixed', 'nominal'], 
             'Utilities':['Type of utilities available', 'mutable', 'nominal'], 
             'LotConfig':['Lot configuration', 'fixed', 'nominal'],
             'LandSlope':['Slope of property', 'fixed', 'nominal'], 
             'Neighborhood':['Physical locations within Ames city limits', 'fixed', 'nominal'], 
             'Condition1':['Proximity to various conditions', 'fixed', 'nominal'], 
             'Condition2':['Proximity to various conditions (if more than one is present)', 'fixed', 'nominal'], 
             'BldgType':['Type of dwelling', 'fixed', 'nominal'],
             'HouseStyle':['Style of dwelling', 'fixed', 'nominal'], 
             'OverallQual':['Rates the overall material and finish of the house', 'mutable', 'ordinal'], 
             'OverallCond':['Rates the overall condition of the house','mutable', 'ordinal'], 
             'YearBuilt':['Original construction date', 'fixed', 'discrete'], 
             'YearRemodAdd':['Remodel date (same as construction date if no remodeling or additions)', 'fixed', 'discrete'],
             'RoofStyle':['Type of roof', 'fixed', 'nominal'], 
             'RoofMatl':['Roof material', 'mutable', 'nominal'], 
             'Exterior1st':['Exterior covering on house', 'mutable', 'nominal'], 
             'Exterior2nd':['Exterior covering on house (if more than one material)', 'mutable', 'nominal'], 
             'MasVnrType':['Masonry veneer type', 'mutable', 'nominal'],
             'MasVnrArea':['Masonry veneer area in square feet', 'mutable', 'continuous'], 
             'ExterQual':['Evaluates the quality of the material on the exterior', 'mutable', 'nominal'], 
             'ExterCond':['Evaluates the present condition of the material on the exterior', 'mutable', 'nominal'], 
             'Foundation':['Type of foundation', 'fixed', 'nominal'], 
             'BsmtQual':['Evaluates the height of the basement', 'mutable', 'nominal'],
             'BsmtCond':['Evaluates the general condition of the basement', 'mutable', 'nominal'], 
             'BsmtExposure':['Refers to walkout or garden level walls', 'fixed', 'nominal'], 
             'BsmtFinType1':['Rating of basement finished area', 'mutable', 'nominal'], 
             'BsmtFinSF1':['Type 1 finished square feet', 'fixed', 'continuous'],
             'BsmtFinType2':['Rating of basement finished area (if multiple types)', 'mutable', 'nominal'], 
             'BsmtFinSF2':['Type 2 finished square feet', 'fixed', 'continuous'], 
             'BsmtUnfSF':['Unfinished square feet of basement area', 'fixed', 'continuous'], 
             'TotalBsmtSF':['Total square feet of basement area', 'fixed', 'continuous'], 
             'Heating':['Type of heating', 'mutable', 'nominal'],
             'HeatingQC':['Heating quality and condition', 'mutable', 'nominal'], 
             'CentralAir':['Central air conditioning', 'mutable', 'ordinal'], 
             'Electrical':['Electrical system', 'mutable', 'nominal'], 
             '1stFlrSF':['First Floor square feet','fixed', 'continuous'], 
             '2ndFlrSF':['Second floor square feet','fixed', 'continuous'],
             'LowQualFinSF':['Low quality finished square feet (all floors)', 'mutable', 'continuous'], 
             'GrLivArea':['Above grade (ground) living area square feet', 'fixed', 'continuous'], 
             'BsmtFullBath':['Basement full bathrooms', 'mutable', 'discrete'], 
             'BsmtHalfBath':['Basement half bathrooms', 'mutable', 'discrete'], 
             'FullBath':['Full bathrooms above grade', 'mutable', 'discrete'],
             'HalfBath':['Half baths above grade', 'mutable', 'discrete'], 
             'BedroomAbvGr':['Bedrooms above grade (does NOT include basement bedrooms)', 'mutable', 'discrete'], 
             'KitchenAbvGr':['Kitchens above grade', 'mutable', 'discrete'], 
             'KitchenQual':['Kitchen quality', 'mutable', 'nominal'],
             'TotRmsAbvGrd':['Total rooms above grade (does not include bathrooms)', 'fixed', 'discrete'], 
             'Functional':['Home functionality (Assume typical unless deductions are warranted)', 'mutable', 'nominal'], 
             'Fireplaces':['Number of fireplaces', 'fixed', 'discrete'], 
             'FireplaceQu':['Fireplace quality', 'mutable', 'nominal'], 
             'GarageType':['Garage location','fixed','nominal'],
             'GarageYrBlt':['Year garage was built','fixed','discrete'], 
             'GarageFinish':['Interior finish of the garage','mutable','nominal'], 
             'GarageCars':['Size of garage in car capacity','fixed','discrete'], 
             'GarageArea':['Size of garage in square feet','fixed','continuous'], 
             'GarageQual':['Garage quality','mutable','nominal'],
             'GarageCond':['Garage condition','mutable','nominal'], 
             'PavedDrive':['Paved driveway','mutable','nominal'], 
             'WoodDeckSF':['Wood deck area in square feet','fixed','continuous'], 
             'OpenPorchSF':['Open porch area in square feet','fixed','continuous'],
             'EnclosedPorch':['Enclosed porch area in square feet','fixed','continuous'], 
             '3SsnPorch':['Three season porch area in square feet','fixed','continuous'], 
             'ScreenPorch':['Screen porch area in square feet','fixed','continuous'], 
             'PoolArea':['Pool area in square feet','fixed','continuous'], 
             'PoolQC':['Pool quality','mutable','nominal'],
             'Fence':['Fence quality','mutable','nominal'], 
             'MiscFeature':['Miscellaneous feature not covered in other categories','fixed','nominal'], 
             'MiscVal':['$Value of miscellaneous feature','fixed','continuous'], 
             'MoSold':['Month Sold (MM)','fixed','discrete'], 
             'YrSold':['Year Sold (YYYY)','fixed','discrete'], 
             'SaleType':['Type of sale','fixed','nominal'],
             'SaleCondition':['Condition of sale','fixed','nominal'], 
             'SalePrice':['Value of property at sale','fixed','cont-price']}

## Data Cleaning and Processing

In [13]:
# check MSZoning column
house['MSZoning'].value_counts()

RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64

In [14]:
# remove non-residential properties from the dataset
house = house[house['MSZoning'] != 'FV']
house = house[house['MSZoning'] != 'C (all)']

In [15]:
house['MSZoning'].value_counts()

RL    1151
RM     218
RH      16
Name: MSZoning, dtype: int64

In [16]:
house.shape

(1385, 80)

In [17]:
house.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1385 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1385 non-null   int64  
 1   MSZoning       1385 non-null   object 
 2   LotFrontage    1134 non-null   float64
 3   LotArea        1385 non-null   int64  
 4   Street         1385 non-null   object 
 5   Alley          65 non-null     object 
 6   LotShape       1385 non-null   object 
 7   LandContour    1385 non-null   object 
 8   Utilities      1385 non-null   object 
 9   LotConfig      1385 non-null   object 
 10  LandSlope      1385 non-null   object 
 11  Neighborhood   1385 non-null   object 
 12  Condition1     1385 non-null   object 
 13  Condition2     1385 non-null   object 
 14  BldgType       1385 non-null   object 
 15  HouseStyle     1385 non-null   object 
 16  OverallQual    1385 non-null   int64  
 17  OverallCond    1385 non-null   int64  
 18  YearBuil

In [20]:
# check Alley column.
house.Alley.value_counts()

Grvl    50
Pave    15
Name: Alley, dtype: int64

In [21]:
len(house[house['Alley']=='NA'])

0

In [22]:
len(house[house['Alley']==np.nan])

0

In [19]:
# check FireplaceQu column.
house.FireplaceQu.value_counts()

Gd    363
TA    306
Fa     32
Ex     23
Po     20
Name: FireplaceQu, dtype: int64

In [23]:
len(house[house['FireplaceQu']=='NA'])

0

In [25]:
len(house[house['FireplaceQu']==np.nan])

0

In [20]:
# check PoolQC column.
house.PoolQC.value_counts()

Gd    3
Ex    2
Fa    2
Name: PoolQC, dtype: int64

In [21]:
# check Fence column.
house.Fence.value_counts()

MnPrv    155
GdPrv     59
GdWo      52
MnWw      11
Name: Fence, dtype: int64

In [22]:
# check MiscFeature column.

house.MiscFeature.value_counts()

Shed    47
Gar2     2
Othr     2
TenC     1
Name: MiscFeature, dtype: int64

In [ ]:
# Some features seem to have a lot of missing values (NaN).
# However the data description indicates that 'NA' means the property does not have a particular feature, rather than being a
# missing value.
# I will replace the 'NA' values with 'No' so they will be not counted as missing values.




In [23]:
# replace NA with No

Cols_using_NA = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType',
           'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']

house[Cols_using_NA] = house[Cols_using_NA].replace({'NA': 'No', np.nan: 'No'})


In [24]:
house.groupby('Alley')['SalePrice'].median().sort_values()

Alley
Grvl    119500.0
Pave    159434.0
No      163250.0
Name: SalePrice, dtype: float64

In [25]:
house.groupby('Alley')['SalePrice'].mean().sort_values()

Alley
Grvl    122219.080000
Pave    168127.533333
No      182466.579545
Name: SalePrice, dtype: float64

In [26]:
house.groupby('BsmtQual')['SalePrice'].median().sort_values()

BsmtQual
No    101800.0
Fa    112000.0
TA    136500.0
Gd    192000.0
Ex    318000.0
Name: SalePrice, dtype: float64

In [27]:
house.groupby('BsmtQual')['SalePrice'].mean().sort_values()

BsmtQual
No    105652.891892
Fa    115692.028571
TA    141796.309859
Gd    202551.825312
Ex    330008.477876
Name: SalePrice, dtype: float64

In [28]:
house.groupby('BsmtCond')['SalePrice'].median().sort_values()

BsmtCond
Po     64000.0
No    101800.0
Fa    118500.0
TA    163000.0
Gd    192140.0
Name: SalePrice, dtype: float64

In [29]:
house.groupby('BsmtCond')['SalePrice'].mean().sort_values()

BsmtCond
Po     64000.000000
No    105652.891892
Fa    122384.395349
TA    182875.460548
Gd    214061.377049
Name: SalePrice, dtype: float64

In [30]:
house.groupby('BsmtExposure')['SalePrice'].mean().sort_values()

BsmtExposure
No    161983.342246
Mn    190677.462264
Av    206152.771429
Gd    257689.805970
Name: SalePrice, dtype: float64

In [31]:
house.groupby('BsmtExposure')['SalePrice'].median().sort_values()

BsmtExposure
No    148000.0
Mn    174000.0
Av    185250.0
Gd    226975.0
Name: SalePrice, dtype: float64

In [32]:
house.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1385 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1385 non-null   int64  
 1   MSZoning       1385 non-null   object 
 2   LotFrontage    1134 non-null   float64
 3   LotArea        1385 non-null   int64  
 4   Street         1385 non-null   object 
 5   Alley          1385 non-null   object 
 6   LotShape       1385 non-null   object 
 7   LandContour    1385 non-null   object 
 8   Utilities      1385 non-null   object 
 9   LotConfig      1385 non-null   object 
 10  LandSlope      1385 non-null   object 
 11  Neighborhood   1385 non-null   object 
 12  Condition1     1385 non-null   object 
 13  Condition2     1385 non-null   object 
 14  BldgType       1385 non-null   object 
 15  HouseStyle     1385 non-null   object 
 16  OverallQual    1385 non-null   int64  
 17  OverallCond    1385 non-null   int64  
 18  YearBuil

In [ ]:
house.Alley.value_counts()

In [ ]:
# check YearBuilt column

house.YearBuilt.unique()

In [ ]:
# check YrSold column

house.YrSold.unique()

In [ ]:
continuous_vars = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
                   '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
                   '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']

discrete_vars = ['YearBuilt', 'YearRemodAdd', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
                 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'MoSold', 'YrSold']

ordinal_vars = ['OverallQual', 'OverallCond', 'CentralAir']

nominal_vars = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
                'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
                'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
                'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'Electrical', 'KitchenQual', 'Functional',
                'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence',
                'MiscFeature', 'SaleType', 'SaleCondition']

quant_vars = continuous_vars + discrete_vars

categ_vars = ordinal_vars + nominal_vars

price = ['SalePrice']

In [ ]:
# explore correlations between continuous variables and price

plt.figure(figsize=(15, 15))
sns.heatmap(house[continuous_vars+price].corr(), cmap='Blues', annot=True)
plt.show;

In [ ]:
# explore correlations between discrete variables, ordinal variables and price

plt.figure(figsize=(15, 15))
sns.heatmap(house[discrete_vars+ordinal_vars+price].corr(), cmap='Blues', annot=True)
plt.show;

In [ ]:
cont_vars_sub = ['TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'GarageArea']

disc_vars_sub = ['FullBath', 'TotRmsAbvGrd', 'GarageCars', 'OverallQual']

In [ ]:
sns.set_theme(style='white' , font_scale=1.5)

plt.figure(figsize=(15, 15))

sns.pairplot(house[cont_vars_sub+price], kind='reg', height=3.5, plot_kws=dict(marker="o"),diag_kws=dict(fill=True),
             corner=True);


In [ ]:
sns.set_theme(style='white' , font_scale=1.5)

plt.figure(figsize=(15, 15))

sns.pairplot(house[disc_vars_sub+price], kind='reg', height=3.5, plot_kws=dict(marker="o"),diag_kws=dict(fill=True),
             corner=True);


In [ ]:
# drop rows with missing values for MasVnrType, MasVnrArea, Electrical

house = house[(house['MasVnrType'].notnull()) & (house['MasVnrArea'].notnull()) & (house['Electrical'].notnull())]

In [ ]:
house.info()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
null_data = house[house.isnull().any(axis=1)]

In [ ]:
null_data

In [ ]:
# Identify fixed features that can predict price.

"""

For the fixed features I have not only considered the ones that cannot be renovated, but also the ones that are unlikely
to be renovated by the real estate company because of the huge cost involved.


LotFrontage
LotArea
LandContour
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
OverallQual
OverallCond
BsmtQual
TotalBsmtSF
1stFlrSF
2ndFlrSF
GrLivArea
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
TotRmsAbvGrd
Fireplaces
GarageType
GarageArea

"""



In [ ]:
# copy the original dataframe.
# the new dataframe will be used for initial modelling

df = house.copy().reset_index()
df

In [ ]:
from sklearn.impute import KNNImputer

impute_knn = KNNImputer(n_neighbors=2)

In [ ]:
# impute missing values in LotFrontage using the most correlated features

imp_LF = df[['LotFrontage', '1stFlrSF', 'LotArea', 'GrLivArea']].copy()
imp_LF.info()

In [ ]:
imp_LF = impute_knn.fit_transform(imp_LF)

In [ ]:
cols = ['LotFrontage', '1stFlrSF', 'LotArea', 'GrLivArea']

imp_LF = pd.DataFrame(data=imp_LF, columns=cols)
imp_LF

In [ ]:
imp_LF.info()

In [ ]:
# impute missing values in GarageYrBlt using the most correlated features

imp_GY = df[['GarageYrBlt', 'YearBuilt', 'YearRemodAdd', 'GarageCars']].copy()
imp_GY.info()

In [ ]:
imp_GY = impute_knn.fit_transform(imp_GY)

In [ ]:
cols = ['GarageYrBlt', 'YearBuilt', 'YearRemodAdd', 'GarageCars']

imp_GY = pd.DataFrame(data=imp_GY, columns=cols)
imp_GY

In [ ]:
imp_GY.info()

In [ ]:
# replace columns in the original dataframe

df['LotFrontage'] = imp_LF['LotFrontage']
df['GarageYrBlt'] = imp_GY['GarageYrBlt']

In [ ]:
df.info()

In [ ]:
fixed_features = ['LotFrontage', 'LotArea', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
                  'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'BsmtQual', 'TotalBsmtSF',
                  '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
                  'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageType', 'GarageArea']

fixed_cat_vars = [x for x in df.columns if x in fixed_features and x in categ_vars]


In [ ]:
fixed_cat_vars

In [ ]:
other_features = [x for x in df.columns if x not in fixed_features and x != 'SalePrice']

In [ ]:
other_features

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
# dummify the categorical variables

df_dum = pd.get_dummies(df, columns=fixed_cat_vars, drop_first=True)


In [ ]:
df_dum.head()

In [ ]:
# define target variable

y = df_dum.pop('SalePrice')

In [ ]:
# define predictor matrix

X = df_dum.drop(columns=other_features)

In [ ]:
X.head()

In [ ]:
df_dum.YrSold.value_counts()

In [ ]:
# create train and test sets based on YrSold

X_train = X[df_dum.YrSold < 2010]
X_test = X[df_dum.YrSold == 2010]

y_train = y[df_dum.YrSold < 2010]
y_test = y[df_dum.YrSold == 2010]

In [ ]:
# train and test set baseline

print(y_train.value_counts(normalize=True).max())
print(y_test.value_counts(normalize=True).max())

In [ ]:
# rescale the dataset

scaler = StandardScaler()

In [ ]:
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# instatiate the model
model = LinearRegression()

# fit the model
model.fit(X_train, y_train)

# score the model
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))

In [ ]:
print('Intercept:', model.intercept_)

lr_coeffs = pd.DataFrame(list(zip(X_train.columns, model.coef_)))
lr_coeffs

In [ ]:
# best estimate of price

lr_coeffs[lr_coeffs[1]==lr_coeffs[1].max()]

In [ ]:
# best estimates of price

lr_coeffs.nlargest(5,[1])

In [ ]:
# worst estimates of price

lr_coeffs.nsmallest(5,[1])

In [ ]:
# perform 5-fold cross validation

scores = cross_val_score(model, X, y, cv=5)
print("Cross-validated scores:", scores)
print("Mean of Ccoss-validated scores:", scores.mean())

In [ ]:
# A:

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Determine any value of *changeable* property characteristics unexplained by the *fixed* ones.

---

Now that you have a model that estimates the price of a house based on its static characteristics, we can move forward with part 2 and 3 of the plan: what are the costs/benefits of quality, condition, and renovations?

---

**Your goals:**
1. Evaluate the effect in dollars of the renovatable features. 
- How would your company use this second model and its coefficients to determine whether they should buy a property or not? Explain how the company can use the two models you have built to determine if they can make money. 
- Investigate how much of the variance in price remaining is explained by these features.
- Do you trust your model? Should it be used to evaluate which properties to buy and fix up?

In [ ]:
"""

I am going to use the changeable features in the predictor matrix and fit a new model.


"""

In [ ]:
# copy the updated dataframe to create a new predictor matrix

df2 = df.copy().reset_index()
df2

In [ ]:
df2.drop(columns=['level_0', 'index'], inplace=True)
df2

In [ ]:
# A:

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. What property characteristics predict an "abnormal" sale?

---

The `SaleCondition` feature indicates the circumstances of the house sale. From the data file, we can see that the possibilities are:

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)
       
One of the executives at your company has an "in" with higher-ups at the major regional bank. His friends at the bank have made him a proposal: if he can reliably indicate what features, if any, predict "abnormal" sales (foreclosures, short sales, etc.), then in return the bank will give him first dibs on the pre-auction purchase of those properties (at a dirt-cheap price).

He has tasked you with determining (and adequately validating) which features of a property predict this type of sale. 

---

**Your task:**
1. Determine which features predict the `Abnorml` category in the `SaleCondition` feature.
- Justify your results.

This is a challenging task that tests your ability to perform classification analysis in the face of severe class imbalance. You may find that simply running a classifier on the full dataset to predict the category ends up useless: when there is bad class imbalance classifiers often tend to simply guess the majority class.

It is up to you to determine how you will tackle this problem. It is recommended to do some research to find out how others have dealt with the problem in the past. Make sure to justify your solution. Don't worry about it being "the best" solution, but be rigorous.

Be sure to indicate which features are predictive (if any) and whether they are positive or negative predictors of abnormal sales.

In [ ]:
# A: